In [ ]:
import MDAnalysis as mda
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#-----Uploading trajectory file to MDAnalyisis as a Universe----------

u = mda.Universe("FUS.gsd")

In [ ]:
#-----Definiton of the 100 Proteins by Particle IDs-------
proteins = {}

for i in range(0,100):
    proteins[i] = u.select_atoms("index "+str(163*i)+":" +str(163*(i+1)-1))

In [ ]:
#---(ALL FRAMES) Main Loop to Determine High and Low Densities and Calculate Radius of Gyration----

tot_high_den_rog = 0
tot_low_den_rog = 0
num_high_protein = 0
num_low_protein = 0


start_frame = 0
end_frame = 1000
avg_high_den_rog = np.zeros([end_frame-start_frame])
avg_low_den_rog = np.zeros([end_frame-start_frame])
frame = 0

for ts in u.trajectory[start_frame:end_frame] :
  for i in range(0,100):
    bin_coor = proteins[i][80].position[2]
    atom_count = u.select_atoms("prop z>"+str(bin_coor-0.5)+" and prop z<"+str(bin_coor+0.5))
    if np.size(atom_count)>400 :
      tot_high_den_rog = tot_high_den_rog + proteins[i].radius_of_gyration(wrap=True)
      num_high_protein = num_high_protein + 1
    if np.size(atom_count)<70 :
      tot_low_den_rog = tot_low_den_rog + proteins[i].radius_of_gyration(wrap=True)
      num_low_protein = num_low_protein + 1
  avg_high_den_rog[frame] = tot_high_den_rog/num_high_protein
  if num_low_protein == 0 :
    avg_low_den_rog[frame] = 0
  if num_low_protein != 0 :
    avg_low_den_rog[frame] = tot_low_den_rog/num_low_protein
  frame = frame + 1
  tot_high_den_rog = 0
  tot_low_den_rog = 0
  num_high_protein = 0
  num_low_protein = 0 

tot_avg_high_den_rog = 0
tot_avg_low_den_rog = 0
avg_frame = 0
equ_start = 550

for j in range(equ_start,end_frame):
  tot_avg_high_den_rog = tot_avg_high_den_rog + avg_high_den_rog[j]
  tot_avg_low_den_rog = tot_avg_low_den_rog + avg_low_den_rog[j]
  avg_frame = avg_frame + 1

tot_avg_high_den_rog = tot_avg_high_den_rog / avg_frame
tot_avg_low_den_rog = tot_avg_low_den_rog / avg_frame

In [ ]:
#----Saving the Data----

f = open('High_Density_ROG.txt', 'a')
np.savetxt(f,avg_high_den_rog, delimiter=" ")
f.close()

f = open('Low_Density_ROG.txt', 'a')
np.savetxt(f,avg_low_den_rog, delimiter=" ")
f.close()

array_tot_avg_rog = np.zeros([2,1])

array_tot_avg_rog[0,0] = tot_avg_high_den_rog
array_tot_avg_rog[1,0] = tot_avg_low_den_rog

f = open('Total_Avg_ROG.txt', 'a')
np.savetxt(f,array_tot_avg_rog, delimiter=" ")
f.close()

In [ ]:
#---------Plotting---------
frames = np.linspace(1,1201,1200)

ax = plt.subplot(111)
ax.plot(frames, avg_low_den_rog, 'b', lw=2, label=r"$Low Density$")
ax.plot(frames, avg_high_den_rog, 'r', lw=2, label=r"$High Density$")
ax.set_xlabel("Frame")
ax.set_ylabel("radius of gyration $R_G$ (nm)")
ax.legend()
ax.figure.savefig("Rgyr.pdf")
plt.show()

In [ ]:
#----------------------------------------------------------------------------------------------------------------------------

In [ ]:
#---(ONLY ONE FRAME) Main Loop to Determine High and Low Densities and Calculate Radius of Gyration----

u.trajectory[1166]

tot_high_den_rog = 0
tot_low_den_rog = 0
num_high_protein = 0
num_low_protein = 0

for i in range(0,100):
  bin_coor = proteins[i][80].position[2]
  atom_count = u.select_atoms("prop z>"+str(bin_coor-0.5)+" and prop z<"+str(bin_coor+0.5))
  if np.size(atom_count)>400 :
    tot_high_den_rog = tot_high_den_rog + proteins[i].radius_of_gyration(wrap=False)
    num_high_protein = num_high_protein + 1
    print(np.size(atom_count),bin_coor,proteins[i].radius_of_gyration(wrap=False))
  if np.size(atom_count)<70 :
    tot_low_den_rog = tot_low_den_rog + proteins[i].radius_of_gyration(wrap=False)
    num_low_protein = num_low_protein + 1
    print(np.size(atom_count),bin_coor,proteins[i].radius_of_gyration(wrap=False))

avg_high_den_rog = tot_high_den_rog/num_high_protein
avg_low_den_rog = tot_low_den_rog/num_low_protein